# Fase 1 - Extração (Landed)

## Importação das bibliotecas

In [1]:
import os
import io
import requests
import zipfile
import pandas as pd
from datetime import date
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField,StringType, FloatType

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import create_map, struct


In [3]:
LANDED = '../landed/'
os.listdir(LANDED)

['.ipynb_checkpoints', 'land.ipynb']

## Sessão Spark

In [4]:
builder = SparkSession.builder.appName("GASTOS PARLAMENTARES")
builder.config(
    "spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2")
builder.config("spark.speculation", "false")
builder.config("spark.sql.parquet.compression.codec", "gzip")
builder.config("spark.debug.maxToStringFields", "100")
builder.config("spark.driver.memory", "1g")
builder.config("spark.driver.cores", "1")
builder.config("spark.executor-memory", "20g")
builder.config("spark.executor.cores", "4")


builder.master("local[*]")

spark = builder.getOrCreate()
spark

## Função para baixar e descompactar arquivo

In [5]:
def downloader(url, out_path):
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(out_path)
        

## Puxa gastos dos parlamentares dos últimos 5 anos

In [6]:
anos = [str(i) for i in range(date.today().year-4, date.today().year+1)]

In [7]:
anos

['2015', '2016', '2017', '2018', '2019']

In [8]:
pipe_files = spark.sparkContext.parallelize(anos) \
                  .foreach(lambda ano: downloader("http://www.camara.leg.br/cotas/Ano-"+ano+".csv.zip",LANDED))

In [10]:
os.listdir(LANDED)

['.ipynb_checkpoints',
 'Ano-2015.csv',
 'Ano-2016.csv',
 'Ano-2017.csv',
 'Ano-2018.csv',
 'Ano-2019.csv',
 'land.ipynb']